## Banning Transit is missing
[PR](https://github.com/cal-itp/data-analyses/issues/1254)

In [1]:
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

In [2]:
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
import _operators_prep as op_prep

In [5]:
analysis_date_list = rt_dates.y2024_dates + rt_dates.y2023_dates

### Edited crosswalk in `gtfs_funnel`: manually added Districts

In [6]:
SCHED_GCS

'gs://calitp-analytics-data/data-analyses/gtfs_schedule/'

In [7]:
GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

'crosswalk/gtfs_key_organization'

In [8]:
url = "gs://calitp-analytics-data/data-analyses/gtfs_schedule/crosswalk/gtfs_key_organization_2024-10-21.parquet"

In [9]:
crosswalk_oct_31 = pd.read_parquet(url)

In [10]:
crosswalk_oct_31.loc[crosswalk_oct_31.name.str.contains("Ban")]

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,organization_source_record_id,organization_name,itp_id,caltrans_district,counties_served,hq_city,hq_county,is_public_entity,is_publicly_operating,funding_sources,on_demand_vehicles_at_max_service,vehicles_at_max_service,number_of_state_counties,uza_name,density,number_of_counties_with_service,state_admin_funds_expended,service_area_sq_miles,population,service_area_pop,subrecipient_type,primary_uza,reporter_type,organization_type,voms_pt,voms_do,year
139,ebc783bace70899492d6206c352547d6,Banning Pass Schedule,recnAiZYHWBxUwH0F,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3Rmcy9iYW5uaW5nLWNhLXVzL2Jhbm5pbmctY2EtdXMuemlw,recuGkFhN2WXGK67H,City of Banning,NaN,08 - San Bernardino,None,None,None,None,None,None,<NA>,<NA>,<NA>,None,NaN,<NA>,NaN,<NA>,NaN,<NA>,None,None,None,None,<NA>,<NA>,<NA>


In [11]:
url2="gs://calitp-analytics-data/data-analyses/gtfs_schedule/crosswalk/gtfs_key_organization_2024-04-15.parquet"

In [12]:
crosswalk_apr_15 = pd.read_parquet(url2)

In [13]:
crosswalk_apr_15.loc[crosswalk_apr_15.name.str.contains("Ban")].organization_name

123    City of Banning
Name: organization_name, dtype: object

### See why it's not working

In [14]:
# Get cardinal direction for each route
df_sched = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [15]:
df_sched.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date
0,014d0998350083249a9eb310635548c2,10866826,1.00,all_day,Northbound,45.00,0.31,8,0.33,0.00,1.00,0.00,1.00,0.00,0.00,2023-10-11
1,014d0998350083249a9eb310635548c2,10866826,1.00,all_day,Northbound,45.00,0.31,8,0.33,0.00,1.00,0.00,1.00,0.00,0.00,2023-11-15


In [16]:
df_rt_sched = (
        merge_data.concatenate_rt_vs_schedule_by_route_direction(
            analysis_date_list
        ))

In [17]:
df_rt_sched.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,name,service_date,schedule_source_record_id
0,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,all_day,1614,1579,2517.85,1201.00,4596,3438,2,8,11,21,1.83,0.75,0.64,0.63,1.00,1.00,2.10,119.90,Bay Area 511 Marin Schedule,2023-04-12,NaN
1,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,all_day,1672,1652,2326.07,1257.00,4953,4287,0,4,18,22,2.13,0.87,0.72,0.71,1.00,1.00,1.85,105.73,Bay Area 511 Marin Schedule,2023-05-17,NaN


In [18]:
df_rt_sched.loc[df_rt_sched.name.str.contains("Ban")][['name','service_date']].drop_duplicates()

,name,service_date
104110,Banning Pass Schedule,2023-03-15
104111,Banning Pass Schedule,2023-04-12
104112,Banning Pass Schedule,2023-05-17
104113,Banning Pass Schedule,2023-06-14
104114,Banning Pass Schedule,2023-07-12
104115,Banning Pass Schedule,2023-08-15
104116,Banning Pass Schedule,2023-09-13
104117,Banning Pass Schedule,2023-10-11
104118,Banning Pass Schedule,2023-11-15
104119,Banning Pass Schedule,2023-12-13


In [19]:
df_crosswalk = merge_data.concatenate_crosswalk_organization(
        analysis_date_list
    )

In [20]:
df_crosswalk.head(2)

,schedule_gtfs_dataset_key,name,schedule_source_record_id,base64_url,organization_source_record_id,organization_name,caltrans_district,service_date
0,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,rec7EN71rsZxDFxZd,Ventura County Transportation Commission,07 - Los Angeles,2024-01-17
1,1770249a5a2e770ca90628434d4934b1,VCTC GMV Schedule,recrAG7e0oOiR6FiP,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,rec1ErIn9gG1Isk5W,City of Simi Valley,07 - Los Angeles,2024-01-17


In [21]:
df_crosswalk.loc[df_crosswalk.name.str.contains("Ban")][['organization_name','name','service_date']].drop_duplicates()

,organization_name,name,service_date
702,City of Banning,Banning Pass Schedule,2024-04-17
1473,City of Banning,Banning Pass Schedule,2024-08-14
1681,City of Banning,Banning Pass Schedule,2024-09-18
1895,City of Banning,Banning Pass Schedule,2024-10-16


In [28]:
DIGEST_RT_SCHED = GTFS_DATA_DICT.digest_tables.route_schedule_vp 
DIGEST_SEGMENT_SPEEDS = GTFS_DATA_DICT.digest_tables.route_segment_speeds

#### Why isn't Banning showing up despite me updating the crosswalk correctly?

In [29]:
df = pd.read_parquet(
        f"{RT_SCHED_GCS}{DIGEST_RT_SCHED}.parquet"
    )

In [30]:
df.head(1)

,schedule_gtfs_dataset_key,direction_id,time_period,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date,typology,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,schedule_source_record_id_x,sched_rt_category,speed_mph,name,route_long_name,route_short_name,route_combined_name,route_id,schedule_source_record_id_y,base64_url,organization_source_record_id,organization_name,caltrans_district,route_primary_direction
0,014d0998350083249a9eb310635548c2,1.00,all_day,45.00,0.31,8,0.33,0.00,1.00,0.00,1.00,0.00,0.00,2023-10-11,rapid,0,0,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,schedule_only,NaN,SLO Schedule,Broad/Airport/Johnson,1B,1B Broad/Airport/Johnson,10866826,recRIGtFV5uxrTx05,aHR0cHM6Ly9hcHAubWVjYXRyYW4uY29tL3VyYi93cy9mZWVkL2MybDBaVDF6Ykc5MGNtRnVjMmwwTzJOc2FXVnVkRDF6Wld4bU8yVjRjR2x5WlQwN2RIbHdaVDFuZEdaek8ydGxlVDB6WlRNd016TTFPVFJpTVRFMk56QTBOMkl4TmpRd05qQTBaalF3TUdNek16ZGlNMkUxTVRRMA==,reciakGBN1DP9dK9N,San Luis Obispo Regional Transit Authority,05 - San Luis Obispo,Northbound


In [34]:
df.loc[df.name.str.contains("Ban")][['caltrans_district','name','organization_name','service_date']].drop_duplicates()

,caltrans_district,name,organization_name,service_date
211456,None,Banning Pass Schedule,None,2023-03-15
211457,None,Banning Pass Schedule,None,2023-04-12
211458,None,Banning Pass Schedule,None,2023-05-17
211459,None,Banning Pass Schedule,None,2023-06-14
211460,None,Banning Pass Schedule,None,2023-07-12
211461,None,Banning Pass Schedule,None,2023-08-15
211462,None,Banning Pass Schedule,None,2023-09-13
211463,None,Banning Pass Schedule,None,2023-10-11
211464,None,Banning Pass Schedule,None,2023-11-15
211465,None,Banning Pass Schedule,None,2023-12-13


### I also want to edit this so if an agency that consistently produces RT or schedule data misses a month or two, they will still show up on the portfolio